In [2]:
import torch
import torch.nn as nn
from torchvision import transforms, models

import numpy as np 
from pathlib import Path
from PIL import Image
from einops import rearrange
import re

import cv2

import matplotlib.pyplot as plt

In [3]:
def set_table(a):
    table = []
    for i in range(256):
        if i < a:
            table.append(0)
        else:
            table.append(1)
    return table

def get_block_num(path):
    img = Image.open(path).resize((1024, 128), Image.Resampling.LANCZOS)
    tmp = np.array(img.point(set_table(50),'1') ).sum(axis=0) > 50
    feature = ''.join(str(list(tmp.astype(int))).strip('[]').split(', '))
    return len(re.findall('011111', feature))

def split_image(input_path, h_split=2):
    w_split = get_block_num(input_path)
    img = Image.open(input_path)
    w, h = img.size
    pic = np.array(img.resize((w//w_split*w_split, h//h_split*h_split), Image.Resampling.LANCZOS))
    pic_split = rearrange(pic, "(d1 H) (d2 W) -> (d1 d2) H W",  d1 = h_split, d2 = w_split)
    
    return [Image.fromarray(pic) for pic in pic_split]

def bina(img, ratio):
    img[img < ratio] = 0
    img[img >= ratio] = 1
    return img
    
def bina_img(batch_img):
    for i, img in enumerate(batch_img):
        img[1] = bina(img[1], 0.12)
        img[2] = bina(img[2], 0.52)
        
        batch_img[i] = img
    return batch_img

def generate_box(input_path, labels):
    w_split = get_block_num(input_path)
    h_split = 2
    img = Image.open(input_path)
    # w_, h_ = img.size
    w_, h_ = 1024, 64
    W, H = w_//w_split*w_split, h_//h_split*h_split
    pic = np.array(img.resize((W, H), Image.Resampling.LANCZOS))
    
    w, h = W//w_split, H//h_split
    
    plt.figure(figsize=(512, 64))
    plt.imshow(pic, cmap='Greys_r')
    
    
    for i, label in enumerate(labels):
        if label == 0:
            y_num, x_num = i // w_split, i % w_split
            x, y = x_num * w, y_num * h
            ax = plt.gca()
            ax.add_patch(plt.Rectangle((x, y), w, h, color="red", fill=False, linewidth=30))
    plt.show()
    
def initialization():
    data_transforms = transforms.Compose([
        transforms.Resize((800,800)),
        transforms.ToTensor()
    ])
    
    model_ft = models.resnet18(weights=None)
    num_ftrs = model_ft.fc.in_features
    model_ft.fc = nn.Linear(num_ftrs, 2)
    # solution 2
    model_path = './models/resnet_2022_07_22_12_52_29.pt'
    model_ft.load_state_dict({k.replace('module.', ''): v for k, v in torch.load(model_path).items()})
    
    model_ft = model_ft.cuda()
    model_ft.eval()
    print('eval finish')
    
    return model_ft, data_transforms

def adapt_threshold(input_img, ratio):
    
    input_img= cv2.blur(input_img, (5,5))
    
    m = 100
    _, img = cv2.threshold(input_img, ratio, 255, cv2.THRESH_BINARY)
    for gg in range(m):
        flag = 0
        for i in input_img:
            s = ''.join(str(list(i.astype(int))).strip('[]').split(', '))
            if len(re.findall('0255255255', s)) > 3:
                ratio -= 1
                _, img = cv2.threshold(input_img, ratio, 255, cv2.THRESH_BINARY)
                flag = 1
                break
        if flag == 1:
            continue
        
        tmp = img.T
        for i in tmp:
            s = ''.join(str(list(i.astype(int))).strip('[]').split(', '))
            if len(re.findall('0255255255',s)) > 3:
                ratio -= 1
                _, img = cv2.threshold(input_img, ratio, 255, cv2.THRESH_BINARY)
                flag = 1
                break
        if flag == 1:
            continue
        
        return img, ratio
    return img, ratio

def filter_img(img):
    img = img[img.sum(axis=1) > 100000,:]
    img = img[:,img.sum(axis=0) > 100000,]
    
    H, W = img.shape
    
    h, w = H//12, W//6
    
    img = np.array(Image.fromarray(img).resize((w*6, h*12), Image.Resampling.LANCZOS))
    img_p = rearrange(img, "(b1 h) (b2 w) -> (b1 b2) (h w)", b1=12, b2=6)
    
    img_p = img_p / np.array([img_p.mean(axis=1)]*h*w).T * img_p.mean()
    
    return rearrange(img_p, "(b1 b2) (h w) -> (b1 h) (b2 w)", b1=12, b2=6, h=h, w=w).astype('uint8')

model_ft, data_transforms = initialization()

root_path = Path.cwd()
fig_path = root_path / "fig"

pic_list = [path for path in fig_path.iterdir() if path.name[-3:] == "png"]

def main(path):
    pics = split_image(path)
    batch_img = torch.cat([torch.cat([data_transforms(p)]*3).unsqueeze(0) for p in pics])
    imgs = bina_img(batch_img)
    output = model_ft(imgs.cuda())
    _, preds = torch.max(output, 1)
    
    damge_values = [] 
    for i, p in zip(preds, pics):
        if i == 1:
            damge_values.append(0)
        else:
            img = filter_img(np.array(p))
            a = 1.3
            img = img * float(a)
            img[img > 255] = 255
            img = np.round(img)

            img = img[img.sum(axis=1) > np.percentile(img.sum(axis=1), 10),:]
            img = img[:,img.sum(axis=0) > np.percentile(img.sum(axis=0), 10)]

            img, ratio = adapt_threshold(img, 200)

            img = (img - 255) * -1

            img = img.astype(np.uint8)
            damge_values.append(img.sum()/(img.shape[0]*img.shape[1]) * 100)
    
    generate_box(pic_list[0], list(preds.cpu().numpy()))
    
    return preds, damge_values     

eval finish


In [13]:
preds, damge_values = main(pic_list[2])

In [14]:
preds

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0,
        1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1], device='cuda:0')